# Data Pre-Processing

## Import packages and libraries

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", 100)
# Create Dataframe
df = pd.read_csv('./data/dataset.csv')
# Print shape of dataset
print(df.shape)

(72946, 10)


# Data cleaning

In [29]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72946 entries, 0 to 72945
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         72946 non-null  float64
 1   high         72946 non-null  float64
 2   low          72946 non-null  float64
 3   close        72946 non-null  float64
 4   volume       72946 non-null  float64
 5   marketCap    72946 non-null  float64
 6   timestamp    72946 non-null  object 
 7   crypto_name  72946 non-null  object 
 8   date         72946 non-null  object 
dtypes: float64(6), object(3)
memory usage: 5.0+ MB


In [31]:
df.isnull().sum()

open           0
high           0
low            0
close          0
volume         0
marketCap      0
timestamp      0
crypto_name    0
date           0
dtype: int64

In [32]:
print(df.duplicated().sum())

0


# FEATURE ENGINNERING

In [33]:
df1=df.copy()
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# Convert 'timestamp' column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)


## 1. Moving Averages

- Description: Moving averages smooth out price data to identify trends, which can help predict volatility shifts.
- Types: Simple Moving Average (SMA) is implied, but Exponential Moving Average (EMA) can also be considered for responsiveness.
- Explanation: These features capture the average closing price over different time windows, smoothing short-term fluctuations and highlighting trends that may precede volatility changes.

In [34]:
# Simple Moving Average (SMA) for 7, 14, and 30 days
df1['SMA_7'] = df1['close'].rolling(window=7).mean()
df1['SMA_14'] = df1['close'].rolling(window=14).mean()
df1['SMA_30'] = df1['close'].rolling(window=30).mean()
df1.head()

,open,high,low,close,volume,marketCap,timestamp,crypto_name,date,SMA_7,SMA_14,SMA_30
0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05,NaN,NaN,NaN
1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05,NaN,NaN,NaN
2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06,NaN,NaN,NaN
3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06,NaN,NaN,NaN
4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07,NaN,NaN,NaN


## 2. Rolling Volatility

- Description: Rolling volatility measures the standard deviation of price changes (typically returns) over a time window, indicating historica volatility.
- Explanation: Rolling volatility quantifies price variability over specified periods, annualized for consistency. Higher values indicate periods of increased price fluctuations, a direct predictor of future volatility.

In [35]:
# Calculate log returns
df1['Log_Return'] = np.log(df1['close'] / df1['close'].shift(1))

# Rolling volatility (standard deviation of log returns) for 7, 14, and 30 days, annualized
df1['Volatility_7'] = df1['Log_Return'].rolling(window=7).std() * np.sqrt(365)
df1['Volatility_14'] = df1['Log_Return'].rolling(window=14).std() * np.sqrt(365)
df1['Volatility_30'] = df1['Log_Return'].rolling(window=30).std() * np.sqrt(365)
df1.head()

,open,high,low,close,volume,marketCap,timestamp,crypto_name,date,SMA_7,SMA_14,SMA_30,Log_Return,Volatility_7,Volatility_14,Volatility_30
0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05,NaN,NaN,NaN,-3.474414,NaN,NaN,NaN
2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06,NaN,NaN,NaN,3.442774,NaN,NaN,NaN
3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06,NaN,NaN,NaN,-3.505890,NaN,NaN,NaN
4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07,NaN,NaN,NaN,3.498741,NaN,NaN,NaN


## 3. Liquidity Ratios

- Description: The liquidity ratio, defined as volume divided by market capitalization, reflects trading activity relative to the cryptocurrency’s market size.
- Explanation: This ratio indicates how liquid a cryptocurrency is. Higher ratios suggest more trading activity, which may correlate with lower volatility due to better price stability, while lower ratios may indicate higher volatility risk.

In [36]:
# Volume-to-Market Cap Ratio
df1['Liquidity_Ratio'] = df1['volume'] / df1['marketCap'] 
df1.head()


,open,high,low,close,volume,marketCap,timestamp,crypto_name,date,SMA_7,SMA_14,SMA_30,Log_Return,Volatility_7,Volatility_14,Volatility_30,Liquidity_Ratio
0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05,NaN,NaN,NaN,-3.474414,NaN,NaN,NaN,0.0
2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06,NaN,NaN,NaN,3.442774,NaN,NaN,NaN,0.0
3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06,NaN,NaN,NaN,-3.505890,NaN,NaN,NaN,0.0
4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07,NaN,NaN,NaN,3.498741,NaN,NaN,NaN,0.0


## 4. Technical Indicators

### 4.1 Bollinger Bands:

- Description: Bollinger Bands consist of a moving average and two bands (upper and lower) set at a multiple of the standard deviation, with the band width indicating volatility.

- Explanation: The Bollinger Band width (difference between upper and lower bands) is a proxy for volatility. Wider bands indicate higher volatility, useful for predicting volatile periods.


In [37]:
# Bollinger Bands (20-day SMA and 2 standard deviations)
df1['SMA_20'] = df1['close'].rolling(window=20).mean()
df1['Std_Dev_20'] = df1['close'].rolling(window=20).std()
df1['Bollinger_Upper'] = df1['SMA_20'] + 2 * df1['Std_Dev_20']
df1['Bollinger_Lower'] = df1['SMA_20'] - 2 * df1['Std_Dev_20']
df1['Bollinger_Width'] = df1['Bollinger_Upper'] - df1['Bollinger_Lower']
df1.head()

,open,high,low,close,volume,marketCap,timestamp,crypto_name,date,SMA_7,SMA_14,SMA_30,Log_Return,Volatility_7,Volatility_14,Volatility_30,Liquidity_Ratio,SMA_20,Std_Dev_20,Bollinger_Upper,Bollinger_Lower,Bollinger_Width
0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05,NaN,NaN,NaN,-3.474414,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06,NaN,NaN,NaN,3.442774,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06,NaN,NaN,NaN,-3.505890,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07,NaN,NaN,NaN,3.498741,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


### 4.2 Average True Range (ATR):

- Description: ATR measures the average range between high and low prices over a period, capturing intraday price volatility.
- Explanation: ATR quantifies the average price range, providing a robust measure of volatility. Higher ATR values indicate larger price swings, useful for volatility forecasting.

In [38]:
# True Range (TR) calculation
df1['TR'] = np.maximum(df1['high'] - df1['low'], 
                      np.maximum(abs(df1['high'] - df1['close'].shift(1)), 
                                 abs(df1['low'] - df1['close'].shift(1))))
# ATR for 14 days
df1['ATR_14'] = df1['TR'].rolling(window=14).mean()
df1.head()

,open,high,low,close,volume,marketCap,timestamp,crypto_name,date,SMA_7,SMA_14,SMA_30,Log_Return,Volatility_7,Volatility_14,Volatility_30,Liquidity_Ratio,SMA_20,Std_Dev_20,Bollinger_Upper,Bollinger_Lower,Bollinger_Width,TR,ATR_14
0,112.900002,118.800003,107.142998,115.910004,0.0,1.288693e+09,2013-05-05T23:59:59.999Z,Bitcoin,2013-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.493130,3.692460,3.346060,3.590890,0.0,6.229819e+07,2013-05-05T23:59:59.999Z,Litecoin,2013-05-05,NaN,NaN,NaN,-3.474414,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,112.563944,NaN
2,115.980003,124.663002,106.639999,112.300003,0.0,1.249023e+09,2013-05-06T23:59:59.999Z,Bitcoin,2013-05-06,NaN,NaN,NaN,3.442774,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,121.072112,NaN
3,3.594220,3.781020,3.116020,3.371250,0.0,5.859436e+07,2013-05-06T23:59:59.999Z,Litecoin,2013-05-06,NaN,NaN,NaN,-3.505890,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,109.183983,NaN
4,112.250000,113.444000,97.699997,111.500000,0.0,1.240594e+09,2013-05-07T23:59:59.999Z,Bitcoin,2013-05-07,NaN,NaN,NaN,3.498741,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,110.072750,NaN


In [39]:
df1.isnull().sum()

open                0
high                0
low                 0
close               0
volume              0
marketCap           0
timestamp           0
crypto_name         0
date                0
SMA_7               6
SMA_14             13
SMA_30             29
Log_Return          1
Volatility_7        7
Volatility_14      14
Volatility_30      30
Liquidity_Ratio    58
SMA_20             19
Std_Dev_20         19
Bollinger_Upper    19
Bollinger_Lower    19
Bollinger_Width    19
TR                  1
ATR_14             14
dtype: int64

In [40]:
df1['Volatility_7_target'] = df1['Volatility_7'].shift(-1)

# Lag features to avoid leakage
lag_features = ['Log_Return', 'Volatility_7', 'Volatility_14', 'Volatility_30',
                'ATR_14', 'TR', 'Bollinger_Width', 'Liquidity_Ratio']
df1[lag_features] = df1[lag_features].shift(1)

## FEATURE SELECTION

In [41]:
# Drop irrelevant features (volume/marketCap kept for now)
irrelevant_features = ['timestamp', 'SMA_7', 'SMA_14', 'SMA_30', 
                       'SMA_20', 'Std_Dev_20', 'Bollinger_Upper', 'Bollinger_Lower']
df1.drop(columns=irrelevant_features, errors='ignore', inplace=True)

# Extract time features
df1['date'] = pd.to_datetime(df1['date'], format='%Y-%m-%d')
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month
df1['day'] = df1['date'].dt.day
df1['day_of_week'] = df1['date'].dt.dayofweek





### Type of Features

In [42]:
num_features = [feature for feature in df1.columns if df1[feature].dtype != 'O']
print('Num of Numerical Features :', len(num_features))

Num of Numerical Features : 20


In [43]:
cat_features = [feature for feature in df1.columns if df1[feature].dtype == 'O']
print('Num of Categorical Features :', len(cat_features))

Num of Categorical Features : 1


In [44]:
discrete_features=[feature for feature in num_features if len(df1[feature].unique())<=25]
print('Num of Discrete Features :',len(discrete_features))

Num of Discrete Features : 3


In [45]:
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print('Num of Continuous Features :',len(continuous_features))

Num of Continuous Features : 17


## feature transfromation


In [46]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin



# Custom transformer to replace inf/-inf with NaN
class InfToNanTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = pd.DataFrame(X).replace([np.inf, -np.inf], np.nan)
        return X.to_numpy()  # ensure numpy array output

# Copy dataset
data = df1.copy()

# Define column types
categorical_features = ['crypto_name']
numeric_features = [
    'open', 'high', 'low', 'close', 'volume', 'marketCap',
    'Log_Return', 'Volatility_7', 'Volatility_14', 'Volatility_30',
    'Liquidity_Ratio', 'Bollinger_Width', 'TR', 'ATR_14'
]

# Pipeline for numeric columns
numeric_transformer = Pipeline(steps=[
    ('inf_to_nan', InfToNanTransformer()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # dense output
])

# Combine into ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Full pipeline
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Separate features & target
X = data.drop(columns=['Volatility_7_target'])

data['Volatility_7_target'] = data['Volatility_7_target'].fillna(method='ffill')

# If still missing (e.g., first row), fill with 0 or mean
data['Volatility_7_target'] = data['Volatility_7_target'].fillna(data['Volatility_7_target'].mean())
y = data['Volatility_7_target']

# Fit-transform
X_processed = full_pipeline.fit_transform(X)

# Retrieve feature names
num_features_out = numeric_features
cat_features_out = list(full_pipeline.named_steps['preprocessor']
                        .named_transformers_['cat']
                        .named_steps['onehot']
                        .get_feature_names_out(categorical_features))

all_feature_names = num_features_out + cat_features_out

# Convert to DataFrame
X_processed_df = pd.DataFrame(X_processed, columns=all_feature_names)

# Add target column back
final_df = pd.concat([X_processed_df, y.reset_index(drop=True),df1[[ 'year', 'month', 'day', 'day_of_week']]], axis=1)
final_df.drop(columns=['Volatility_7'], inplace=True)

# Save to CSV
final_df.to_csv("processed_crypto_with_target.csv", index=False)

print(" Saved as 'processed_crypto_with_target.csv'")
print("Final shape:", final_df.shape)


 Saved as 'processed_crypto_with_target.csv'
Final shape: (72946, 74)


In [50]:
final_df.head(20)

,open,high,low,close,volume,marketCap,Log_Return,Volatility_14,Volatility_30,Liquidity_Ratio,Bollinger_Width,TR,ATR_14,crypto_name_Aave,crypto_name_Algorand,crypto_name_ApeCoin,crypto_name_Aptos,crypto_name_Avalanche,crypto_name_BNB,crypto_name_Basic Attention Token,crypto_name_Binance USD,crypto_name_Bitcoin,crypto_name_Bitcoin Cash,crypto_name_Cardano,crypto_name_Casper,crypto_name_Chain,crypto_name_Chainlink,crypto_name_Chiliz,crypto_name_Cosmos,crypto_name_Cronos,crypto_name_Dai,crypto_name_Decentraland,crypto_name_Dogecoin,crypto_name_EOS,crypto_name_Elrond,crypto_name_Ethereum,crypto_name_Ethereum Classic,crypto_name_FTX Token,crypto_name_Filecoin,crypto_name_Flow,crypto_name_Hedera,crypto_name_Huobi Token,crypto_name_Internet Computer,crypto_name_Litecoin,crypto_name_Maker,crypto_name_Monero,crypto_name_NEAR Protocol,crypto_name_OKB,crypto_name_Polkadot,crypto_name_Polygon,crypto_name_Quant,crypto_name_Ravencoin,crypto_name_Shiba Inu,crypto_name_Solana,crypto_name_Stellar,crypto_name_THORChain,crypto_name_TRON,crypto_name_Terra Classic,crypto_name_Tether,crypto_name_Tezos,crypto_name_The Sandbox,crypto_name_Theta Network,crypto_name_Toncoin,crypto_name_UNUS SED LEO,crypto_name_USD Coin,crypto_name_Uniswap,crypto_name_VeChain,crypto_name_Wrapped Bitcoin,crypto_name_XRP,Volatility_7_target,year,month,day,day_of_week
0,-0.144753,-0.144040,-0.145081,-0.144282,-0.229533,-0.179447,0.000299,-0.112827,-0.147954,-0.243948,-0.38348,-0.231043,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,5,6
1,-0.165666,-0.165362,-0.165516,-0.165736,-0.229533,-0.195797,0.000299,-0.112827,-0.147954,-0.353371,-0.38348,-0.231043,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,5,6
2,-0.144165,-0.142954,-0.145180,-0.144972,-0.229533,-0.179976,-0.600218,-0.112827,-0.147954,-0.353371,-0.38348,-0.217222,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,6,0
3,-0.165647,-0.165346,-0.165561,-0.165778,-0.229533,-0.195846,0.594814,-0.112827,-0.147954,-0.353371,-0.38348,-0.216059,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,6,0
4,-0.144878,-0.145032,-0.146940,-0.145124,-0.229533,-0.180088,-0.605656,-0.112827,-0.147954,-0.353371,-0.38348,-0.217683,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,7,1
5,-0.165689,-0.165415,-0.165596,-0.165785,-0.229533,-0.195853,0.604483,-0.112827,-0.147954,-0.353371,-0.38348,-0.217562,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.293336,2013,5,7,1
6,-0.165706,-0.165399,-0.165528,-0.165770,-0.229533,-0.195834,-0.606406,-0.112827,-0.147954,-0.353371,-0.38348,-0.217769,-0.418482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.671597,2013,5,8,2
7,-0.145384,-0.144600,-